In [48]:
import os
from openpyxl import load_workbook
import time

### Подключаемся к обезличенному набору данных
Нужно заполнить таблицу по примеру, который показан ниже.


| Номер телефона                   | Скоринговый балл |
|----------------------------------|------------------|
| c210baab81dbbfd3a7f43fb869069fe7 | 0,731            |

Если тестирования проводится на новом наборе данных, нужно точно указать наши 3 новых номеров телефона для нахождения соли.

```jupyterpython
t_num = [89106881090,89057801765,89809229343]
```
Нужно сменит t_num на актуальные номера телефона

In [49]:
md5_list = open('hashcat/md5_list.txt', "w")
wb = load_workbook("scoring_data_v.0.3.3.xlsx")
ws = wb["A2"]
# Oтбор соли с существующими 3 номерами телефона
t_num = [89106881090,89057801765,89809229343]

### Набор обезличенных данных преобразуем в текстовый документ для быстрого перебора

In [50]:
for i in range(2,len(ws["A"])+1):
    md5_list.write(f"{ws['A' + str(i)].value}\n")
md5_list.close()

### Проверка текстового документа на наличие заполнения

In [51]:
read_md5_list = open("hashcat/md5_list.txt", "r")
print(f"len = {bool(read_md5_list)}")

len = True


### Вызываем библиотеку hashcat. Перебор данных осуществляется с помощью масок с длиной 11 символов

In [ ]:
start_bf = time.time()
if bool(read_md5_list) is True:
    os.system("rm -rf ./hashcat/hashcat.potfile")
    os.system("./hashcat/hashcat.bin -m0 -a3 ./hashcat/md5_list.txt ?d?d?d?d?d?d?d?d?d?d?d")
end_bf = time.time()

#### Примерный отчет по брут форсу MD5
```shell
Session..........: hashcat
Status...........: Cracked
Hash.Mode........: 0 (MD5)
Hash.Target......: md5_list.txt
Time.Started.....: Sat Jul 30 23:10:54 2022 (45 secs)
Time.Estimated...: Sat Jul 30 23:11:39 2022 (0 secs)
Kernel.Feature...: Pure Kernel
Guess.Mask.......: ?d?d?d?d?d?d?d?d?d?d?d [11]
Guess.Queue......: 1/1 (100.00%)
Speed.#1.........:  2183.2 MH/s (0.21ms) @ Accel:256 Loops:125 Thr:32 Vec:1
Recovered........: 494936/494936 (100.00%) Digests
Remaining........: 0 (0.00%) Digests
Recovered/Time...: CUR:N/A,N/A,N/A AVG:N/A,N/A,N/A (Min,Hour,Day)
Progress.........: 100000000000/100000000000 (100.00%)
Rejected.........: 0/100000000000 (0.00%)
Restore.Point....: 99996664/100000000 (100.00%)
Restore.Sub.#1...: Salt:0 Amplifier:875-1000 Iteration:0-125
Candidate.Engine.: Device Generator
Candidates.#1....: 46833549496 -> 68874949496
Hardware.Mon.#1..: Temp: 52c Fan: 28% Util: 90% Core:1980MHz Mem:5000MHz Bus:16

Started: Sat Jul 30 23:10:51 2022
Stopped: Sat Jul 30 23:11:40 2022
```

In [53]:
print("all cracked hash in hashcat.profile")

all cracked hash in hashcat.profile


### Oтбор соли с существующими 3 номерами телефона


In [54]:
potfile = open("hashcat/hashcat.potfile", "r")
dict_md5 = {}
array_salts_num = []

### Ф-я соли

In [55]:
def salts(t_num:int,array_salts_num):
    salt = []
    for salts_num in array_salts_num:
        salt.append(salts_num-t_num)
    return salt

### Создания словаря md5

In [56]:
for md5 in potfile:
    array_salts_num.append(int(md5[33:44]))
    dict_md5[str(md5[:32])] = md5[33:44]

### На примеры 3 номеров, выявляем истинную соль

In [57]:
start_salt = time.time()
salts1 = salts(t_num[0],array_salts_num)
salts2 = salts(t_num[1],array_salts_num)
salts3 = salts(t_num[2],array_salts_num)
salt_res = set(salts1) & set(salts2) & set(salts3)
end_salt = time.time()
print(f"salt[{len(salt_res)}] = {sorted(salt_res)}")

salt[1] = [4992064728]


### Деобезличенный набор данных хранится в scoring_data_v.1.0.xlsx

In [58]:
for row in range(2,len(ws["A"])+1):
    if dict_md5[str(ws['A' + str(row)].value)]:
          ws['A' + str(row)] = int(dict_md5[str(ws['A' + str(row)].value)])-list(salt_res)[0]
          ws['C' + str(row)] = list(salt_res)[0]
ws['C1'] = "Соль"
ws['E2'] = f"Расчетное время по brute force = {(round(end_bf - start_bf,2))} сек"
ws['E3'] = f"Расчетное время по соли = {round(end_salt - start_salt,2)} сек"
print(f"Расчетное время по brute force = {round(end_bf - start_bf,2)} сек")
print(f"Расчетное время по соли = {round(end_salt - start_salt,2)} сек")

wb.save(filename="scoring_data_v.1.0.xlsx")
print("Деобезличенный набор данных хранится в scoring_data_v.1.0.xlsx")

os.system("libreoffice --writer scoring_data_v.1.0.xlsx")

Расчетное время по brute force = 49.49 сек
Расчетное время по соли = 0.18 сек
Деобезличенный набор данных хранится в scoring_data_v.1.0.xlsx


0